# 数据加载、存储与文件格式

## 目录
+ 读写文本格式的数据
    + 索引
        + 列名
        + 行索引
    + 非固定分隔符
    + 不规则数据问题
        + 跳过一些行
    + 类型推断和数据转换
        + 自动标记缺失值
        + 手动标记缺失值
        + 字典标记缺失值
    + 迭代
    + 将数据写出到文本格式
        + DataFrame
        + Series
    + 手工处理分隔符格式
        + 标准流程：对不等长左对齐的表格进行处理
        + 定义csv.Dialect的一个子类
        + 将csv语支的参数提供给csv.reader
        + 手工输出分隔符文件
    + JSON数据
    + XML和HTML：Web信息收集


+ 二进制数据格式
    + 使用pickle格式
    + 使用HDF5格式
    + 使用HTML和Web API


+ 使用数据库
    + 将数据库中的数据表手工导入DataFrame
    + 将数据库中的数据表自动导入DataFrame

In [1]:
from numpy.random import randn
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
from pandas import Series, DataFrame
import pandas as pd

## 读写文本格式的数据

pandas中的解析函数
+ read_csv  
从文件、URL、文件型对象中加载带分隔符的数据。默认分隔符为逗号
+ read_table  
从文件、URL、文件型对象中加载带分隔符的数据。默认分隔符为制表符（"\t"）
+ read_fwf  
读取定宽列格式数据（也就是说，没有分隔符）
+ read_clipboard  
读取剪贴板中的数据，可以看做read_table的剪贴板版。在将网页转换为表格时很有用

解析函数的选项
+ 索引  
将一个或多个列当做返回的DataFrame处理，以及是否从文件、用户读取列名
+ 类型推断和数据转换  
包括用户定义值的转换、缺失值标记列表等
+ 日期解析  
包括组合功能，比如将分散在多个列中的日期时间信息组合成结果中的单个列
+ 迭代  
支持对大文件进行逐块迭代
+ 不规则数据问题  
跳过一些行、页脚、注释或其他一些不重要的东西（比如由成千上万个逗号隔开的数值数据）

read_csv/read_table函数的参数
+ path  
表示文件系统位置、URL、文件型对象的字符串
+ sep或delimiter  
用于对行中各字段进行拆分的字符序列或正则表达式
+ header  
用作列明的行号。默认为0（第一行），如果没有header行就应该设置为None
+ index_col  
用作行索引的列编号或列名。可以是单个名称/数字或由多个名称/数字组成的列表（层次化索引）
+ names  
用于结果的列名列表，结果header=None
+ skiprows  
需要忽略的行数（从文件开始处算起），或需要跳过的行号列表（从0开始）
+ na_values  
一组用于替换NA的值
+ comment  
用于将注释信息从行尾拆分出去的字符（一个或多个）
+ parse_dates  
尝试将数据解析为日期，默认为False。如果为True，则尝试解析所有列。此外，还可以指定需要解析的一组列号或列名。如果列名的元素为列表或元组，就会将多个列组合到一起再进行日期解析工作（例如，日期/时间分别位于两个列中）
+ keep_date_col  
如果连接多列解析日期，则保持参与连接的列。默认为False
+ converters  
由列号/列名跟函数之间的映射关系组成的字典。例如，{'foo':f}会对foo列的所有值应用函数f
+ dayfirst  
当解析有歧义的日期时，将其看做国际格式（例如，7/6/2012 - June 7, 2012）。默认为False
+ date_parser  
用于解析日期的函数
+ nrows  
需要读取的行数（从文件开始处算起）
+ iterator  
返回一个TextParser以便逐块读取文件
+ chunksize  
文件块的大小（用于迭代）
+ skip_footer  
需要忽略的行数（从文件末尾处算起）
+ verbose  
打印各种解析器输出信息，比如“非数值列中缺失值的数量”等
+ encoding  
用于unicode的文本编码格式。例如，“utf-8”表示用UTF-8编码的文本
+ squeeze  
如果数据经解析后仅含一列，则返回Series
+ thousands  
千分位分隔符，如“,”或“.”

In [2]:
df = pd.read_csv('./data/ch06/ex1.csv') # 以逗号分隔的文件，用read_csv读入一个DataFrame
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [3]:
pd.read_table('./data/ch06/ex1.csv', sep=',') # 也可以用read_table读入，需要指定分隔符

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


### 索引

#### 列名

In [4]:
pd.read_csv('./data/ch06/ex2.csv', header=None) # 无标题行的文件，设置header=None，让pandas为其分配默认的列名

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [5]:
pd.read_csv('./data/ch06/ex2.csv', names=['a', 'b', 'c', 'd', 'message']) # 无标题行的文件，通过names参数自己定义列名

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


#### 行索引

In [6]:
names = ['a', 'b', 'c', 'd', 'message']
pd.read_csv('./data/ch06/ex2.csv', names=names, index_col='message') # 通过index_col参数，将某列做成DataFrame的索引

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [7]:
parsed = pd.read_csv('./data/ch06/csv_mindex.csv', index_col=['key1', 'key2']) # 将多个列做成层次化索引
parsed

value1 
 value2 
 
 
 key1 
 key2 
 
 
 
 
 
 
 one 
 a 
 1 
 2 
 
 
 b 
 3 
 4 
 
 
 c 
 5 
 6 
 
 
 d 
 7 
 8 
 
 
 two 
 a 
 9 
 10 
 
 
 b 
 11 
 12 
 
 
 c 
 13 
 14 
 
 
 d 
 15 
 16

### 非固定分隔符
编写正则表达式来作为read_table的分隔符

In [8]:
list(open('./data/ch06/ex3.txt')) # 各个字段由数量不定的空白符分隔

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

In [9]:
result = pd.read_table('./data/ch06/ex3.txt', sep='\s+') # 用正则表达式\s+表示数量不定的空白分隔符
result # 由于列名比数据行的数量少，read_table推断第一个列是DataFrame的索引

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


### 不规则数据问题

#### 跳过一些行

In [10]:
pd.read_csv('./data/ch06/ex4.csv', skiprows=[0, 2, 3]) # 用skiprows跳过文件的第一行、第三行和第四行

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


### 类型推断和数据转换

#### 自动标记缺失值

In [11]:
result = pd.read_csv('./data/ch06/ex5.csv') # 默认情况下，pandas会用一组经常出现的标记值识别缺失值，如NA、-1.#IND以及NULL等
result

,something,a,b,c,d,message
0,one,1,2,3,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11,12,foo


In [12]:
pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


#### 手动标记缺失值

In [13]:
result = pd.read_csv('./data/ch06/ex5.csv', na_values=['NULL']) # na_values可以接受一组用于表示缺失值的字符串
result

,something,a,b,c,d,message
0,one,1,2,3,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11,12,foo


#### 字典标记缺失值

In [14]:
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}
pd.read_csv('./data/ch06/ex5.csv', na_values=sentinels) # 可以用一个字典，为各列指定不同的NA标记值

,something,a,b,c,d,message
0,one,1,2,3,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11,12,NaN


### 迭代
逐块读取文本文件

In [15]:
result = pd.read_csv('./data/ch06/ex6.csv')
result.info() # 10000行数据

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Data columns (total 5 columns):
one      10000 non-null float64
two      10000 non-null float64
three    10000 non-null float64
four     10000 non-null float64
key      10000 non-null object
dtypes: float64(4), object(1)
memory usage: 468.8+ KB


In [16]:
pd.read_csv('./data/ch06/ex6.csv', nrows=5) # 通过nrows指定读取的行数

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


In [17]:
chunker = pd.read_csv('./data/ch06/ex6.csv', chunksize=1000)# 设置chunksize（行数），逐块读取文件
chunker # 返回TextFileReader对象，可以根据chunksize对文件进行逐块迭代

In [18]:
chunker = pd.read_csv('./data/ch06/ex6.csv', chunksize=1000)

tot = Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0) # fill_value=0，Series相加，重新索引时，指定填充值

tot = tot.sort_values(ascending=False)

In [19]:
tot[:10]

E    368
X    364
L    346
O    343
Q    340
M    338
J    337
F    335
K    334
H    330
dtype: float64

### 将数据写出到文本格式

#### DataFrame

In [20]:
data = pd.read_csv('./data/ch06/ex5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11,12,foo


In [21]:
data.to_csv('./data/ch06/out.csv') # DataFrame的to_csv方法，可以将数据写到一个以逗号分隔的文件中

In [22]:
data.to_csv(sys.stdout, sep='|') # sep参数指定其他分隔符

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


In [23]:
data.to_csv(sys.stdout, na_rep='NULL') # 缺失值在输出结果中默认被表示为空字符串，na_rep参数将缺失值表示为别的标记值

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


In [24]:
data.to_csv(sys.stdout, index=False, header=False) # 默认写出行、列标签，可以设置参数禁用行、列标签

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


In [25]:
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c']) # 可以只写出一部分的列，并以指定的顺序排列

a,b,c
1,2,3.0
5,6,
9,10,11.0


#### Series

In [26]:
dates = pd.date_range('1/1/2000', periods=7)
ts = Series(np.arange(7), index=dates)
ts.to_csv('./data/ch06/tseries.csv')
ts

2000-01-01    0
2000-01-02    1
2000-01-03    2
2000-01-04    3
2000-01-05    4
2000-01-06    5
2000-01-07    6
Freq: D, dtype: int32

In [27]:
Series.from_csv('./data/ch06/tseries.csv', parse_dates=True) #Series的from_csv方法，可以自动设置无header行，第一列做索引，比read_csv方便

2000-01-01    0
2000-01-02    1
2000-01-03    2
2000-01-04    3
2000-01-05    4
2000-01-06    5
2000-01-07    6
dtype: int64

### 手工处理分隔符格式
csv模块，csv.reader函数

CSV语支选项
+ delimiter  
用于分隔字段的单字符字符串。默认为“，”
+ lineterminator  
用于写操作的行结束符，默认为“\r\n”。读操作将忽略此选项，它能认出跨平台的行结束符
+ quotechar  
用于带有特殊字符（如分隔符）的字段的引用符号。默认为“"”
+ quoting  
引用约定。可选值包括csv.QUOTE_ALL（引用所有字段）、csv.QUOTE_MINIMAL（只引用带有诸如分隔符之类特殊字符的字段）、csv.QUOTE_NONNUMERIC以及csv.QUOTE_NON（不引用）。完整信息请参考Python的文档。默认为QUOTE_MINIMAL
+ skipinitialspace  
忽略分隔符后面的空白符。默认为False
+ doublequote  
如何处理字段内的引用符号。如果为True，则双写。完整信息及行为请参见在线文档
+ escapechar  
用于对分隔符进行转移的字符串（如果quoting被设置为csv.QUOTE_NONE的话）。默认禁用

In [28]:
import csv
f = open('./data/ch06/ex7.csv') # 生成句柄

reader = csv.reader(f) # 生成一个迭代器，以原文件中每行为一单位

In [29]:
for line in reader: # 对reader进行迭代，每行产生一个列表
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3', '4']


#### 标准流程：对不等长左对齐的表格进行处理

In [30]:
lines = list(csv.reader(open('./data/ch06/ex7.csv')))
header, values = lines[0], lines[1:]
data_dict = {h: v for h, v in zip(header, zip(*values))} # zip(*values)相当于zip(value[0], value[1],...,value[len(value)-1])
data_df = DataFrame(data_dict)
data_df

,a,b,c
0,1,2,3
1,1,2,3


#### 定义csv.Dialect的一个子类
定义出新格式（如专门的分隔符、字符串引用约定、行结束符等）

In [31]:
class my_dialect(csv.Dialect):
    lineterminator = '\n'
    delimiter = ';'
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL

reader = csv.reader(f, dialect=my_dialect)
reader

#### 将csv语支的参数提供给csv.reader

In [32]:
reader = csv.reader(f, delimiter='|')
reader

#### 手工输出分隔符文件
csv.writer函数，接受一个已打开且可写的文件对象、和csv.reader相同的语支和格式化选项

In [33]:
with open('./data/ch06/mydata.csv', 'w') as f:
    writer = csv.writer(f, dialect=my_dialect)
    writer.writerow(('one', 'two', 'three'))
    writer.writerow(('1', '2', '3'))
    writer.writerow(('4', '5', '6'))
    writer.writerow(('7', '8', '9'))

### JSON数据

In [34]:
obj = """
{"name": "Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 25, "pet": "Zuko"},
              {"name": "Katie", "age": 33, "pet": "Cisco"}]
}
"""

In [35]:
import json
result = json.loads(obj) # 通过json.loads可以将JSON字符串转换成Python形式
result

{u'name': u'Wes',
 u'pet': None,
 u'places_lived': [u'United States', u'Spain', u'Germany'],
 u'siblings': [{u'age': 25, u'name': u'Scott', u'pet': u'Zuko'},
  {u'age': 33, u'name': u'Katie', u'pet': u'Cisco'}]}

In [36]:
asjson = json.dumps(result) # json.dumps可以将Python对象转换成JSON格式

In [37]:
siblings = DataFrame(result['siblings'], columns=['name', 'age']) # 向DataFrame构造器传入一组JSON对象，并选取数据字段的子集
siblings

,name,age
0,Scott,25
1,Katie,33


### XML和HTML：Web信息收集

**NB. The Yahoo! Finance API has changed and this example no longer works**

#### 利用lxml.objectify解析XML

## 二进制数据格式

### 使用pickle格式
实现数据的二进制格式存储最简单的方法之一是使用Python内置的pickle序列化

In [38]:
frame = pd.read_csv('./data/ch06/ex1.csv')
frame
frame.to_pickle('./data/ch06/frame_pickle') # to_pickle方法，将数据以pickle形式保存

In [39]:
pd.read_pickle('./data/ch06/frame_pickle') # read_pickle方法，将数据读回到Python

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


### 使用HDF5格式
Python中的HDF5库有两个API（即PyTables和h5py）
+ h5py  
提供了一种直接而高级的HDF5 API访问接口
+ PyTables  
抽象了HDF5的许多细节，以提供多种灵活的数据容器、表索引、查询功能以及对核外计算技术（out-of-core computation）的某些支持

#### pandas中有一个最小化的类似于字典的HDFStore类，它通过PyTables存储pandas对象

In [40]:
store = pd.HDFStore('./data/mydata.h5')
store['obj1'] = frame
store['obj1_col'] = frame['a']
store

<class 'pandas.io.pytables.HDFStore'>
File path: ./data/mydata.h5
/obj1                frame        (shape->[3,5])
/obj1_col            series       (shape->[3])  

In [41]:
store.close()
os.remove('./data/mydata.h5')

### 使用HTML和Web API
requests包和json包

## 使用数据库

In [42]:
# SQLite数据库（通过Python内置的sqlite3驱动器）
# 创建test表
import sqlite3

query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
 c REAL,        d INTEGER
);"""

con = sqlite3.connect(':memory:')
con.execute(query)
con.commit()

In [43]:
# 插入几行数据
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"

con.executemany(stmt, data)
con.commit()

### 将数据库中的数据表手工导入DataFrame

In [44]:
# 从表中选取数据，返回一个元组列表
cursor = con.execute('select * from test')
rows = cursor.fetchall() # 选取所有行
rows

[(u'Atlanta', u'Georgia', 1.25, 6),
 (u'Tallahassee', u'Florida', 2.6, 3),
 (u'Sacramento', u'California', 1.7, 5)]

In [45]:
# 列名位于游标的description属性中
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [46]:
# 创建DataFrame，注意列名的设置方式
DataFrame(rows, columns=zip(*cursor.description)[0])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


### 将数据库中的数据表自动导入DataFrame

In [47]:
import pandas.io.sql as sql
sql.read_sql('select * from test', con)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5
